In [1]:
# CELL 1 — Install (run once)
!pip install -q sentence-transformers faiss-cpu huggingface-hub transformers accelerate bitsandbytes
print("✓ Install complete")


✓ Install complete


In [2]:
# CELL 1: Imports and device
import os, re, json, random, math
from datetime import datetime
from collections import defaultdict
from typing import List, Dict, Any, Tuple


import numpy as np
import pandas as pd


import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✓ Using device: {DEVICE}')

✓ Using device: cuda


In [3]:
# %%
# CELL 2: Configurable settings
EMBED_MODEL = 'bhavyagiri/InLegal-Sbert'
ZERO_SHOT_MODEL = 'law-ai/InCaseLawBERT'
NER_MODEL = 'law-ai/InLegalBERT'


LLM_PREFERRED = [
    "google/flan-t5-large",
    "soketlabs/pragna-1b",
]

USE_LLM = True

In [4]:
# CELL 3 — Load Mistral-7B-Instruct (4-bit, Colab T4 Safe)

from transformers import AutoTokenizer, AutoModelForCausalLM

print('Loading embedding model...')
embedder = SentenceTransformer(EMBED_MODEL, device=DEVICE)
print('✓ Embedder loaded.')

# ZERO-SHOT CLASSIFIER
try:
    print('Loading zero-shot classifier...')
    zsc = pipeline('zero-shot-classification', model=ZERO_SHOT_MODEL, device=0 if DEVICE=='cuda' else -1)
    print('✓ Zero-shot classifier ready.')
except:
    zsc = None
    print('⚠ Zero-shot classifier unavailable.')

# NER MODEL
try:
    print('Loading NER...')
    ner_pipe = pipeline('ner', model=NER_MODEL, aggregation_strategy='simple', device=0 if DEVICE=='cuda' else -1)
    print('✓ NER ready.')
except:
    ner_pipe = None
    print('⚠ NER unavailable, regex fallback enabled.')

# -----------------------------
# LLM: MISTRAL 7B INSTRUCT
# -----------------------------

LLM_PREFERRED = ["mistralai/Mistral-7B-Instruct-v0.2"]

HAS_LLM = False
LLM_MODEL, LLM_TOKENIZER, LLM_NAME = None, None, None

if USE_LLM:
    for mname in LLM_PREFERRED:
        try:
            print(f"Attempting LLM load: {mname}")

            # Tokenizer
            LLM_TOKENIZER = AutoTokenizer.from_pretrained(mname)
            LLM_TOKENIZER.pad_token = LLM_TOKENIZER.eos_token
            LLM_TOKENIZER.padding_side = "left"

            # 4-bit model load (T4-friendly)
            LLM_MODEL = AutoModelForCausalLM.from_pretrained(
                mname,
                load_in_4bit=True,
                torch_dtype=torch.float16,
                device_map="auto"
            )

            HAS_LLM = True
            LLM_NAME = mname
            print(f"✅ Loaded Mistral 7B Instruct: {mname}")
            break

        except Exception as e:
            print(f"❌ Failed: {mname} — {str(e)[:200]}")

if not HAS_LLM:
    print("⚠ No LLM loaded — using deterministic fallback.")


Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✓ Embedder loaded.
Loading zero-shot classifier...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at law-ai/InCaseLawBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


✓ Zero-shot classifier ready.
Loading NER...


...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

✅ Loaded Mistral 7B Instruct: mistralai/Mistral-7B-Instruct-v0.2


In [5]:
# %%
# CELL 4 — Load Legal Q&A Dataset (Kaggle-Optimized, Always Fresh)

from huggingface_hub import snapshot_download
import os, pandas as pd, json, glob

# 🔧 Change this if you use your own dataset repo
REPO_ID = "Techmaestro369/indian-legal-texts-finetuning"

# Create cache dir (Kaggle: /kaggle/working/ is writable)
DATA_DIR = "./legal_data_cache"
os.makedirs(DATA_DIR, exist_ok=True)

print(f"⬇️  Downloading dataset snapshot from {REPO_ID} ...")
DATA_PATH = snapshot_download(
    repo_id=REPO_ID,
    repo_type="dataset",
    local_dir=DATA_DIR,
    local_dir_use_symlinks=False,
    revision="main"   # ensure latest
)
print("✓ Dataset snapshot downloaded")

# Detect .json files automatically
json_files = glob.glob(os.path.join(DATA_PATH, "*.json"))
if not json_files:
    raise FileNotFoundError("No .json files found in the downloaded dataset.")

documents = []
for path in json_files:
    fname = os.path.basename(path)
    try:
        df = pd.read_json(path, lines=False)
        df["source"] = fname.replace(".json", "")
        for _, row in df.iterrows():
            documents.append({
                "id": len(documents),
                "question": str(row.get("question", ""))[:1500],
                "answer": str(row.get("answer", ""))[:8000],
                "text": f"Q: {row.get('question', '')}\nA: {row.get('answer', '')}",
                "source": row.get("source", fname.replace(".json", ""))
            })
        print(f"✓ Loaded {fname}: {len(df)} pairs")
    except Exception as e:
        print(f"⚠️  Error reading {fname}: {e}")

print(f"✅ Total documents prepared: {len(documents)}")

# Optional quick check
if len(documents) > 0:
    print("Sample document:", documents[0])
else:
    print("⚠️ No documents loaded — check repo or format.")


⬇️  Downloading dataset snapshot from Techmaestro369/indian-legal-texts-finetuning ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

✓ Dataset snapshot downloaded
✓ Loaded ipc_qa.json: 2267 pairs
✓ Loaded crpc_qa.json: 8194 pairs
⚠️  Error reading dataset_card.json: All arrays must be of the same length
✓ Loaded constitution_qa.json: 4082 pairs
✅ Total documents prepared: 14543
Sample document: {'id': 0, 'question': 'What is the title and extent of operation of the Indian Penal Code?', 'answer': "The title is 'The Indian Penal Code' and its operation extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.", 'text': "Q: What is the title and extent of operation of the Indian Penal Code?\nA: The title is 'The Indian Penal Code' and its operation extends to the punishment of offences committed within India, and beyond but which by law may be tried within India. It also includes extension of the Code to extra-territorial offences.", 'source': 'ipc_qa'}


In [6]:
# CELL 5 — FAISS

if len(documents) > 0:
    print('Building FAISS index...')
    texts = [d['text'][:256] for d in documents]   # ✅ RAM Safe
    embeddings = embedder.encode(texts, show_progress_bar=True, convert_to_numpy=True).astype('float32')

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    print(f'✓ Vector DB ready. {len(documents)} docs indexed (dim={dim})')

else:
    index = None


Building FAISS index...


Batches:   0%|          | 0/455 [00:00<?, ?it/s]

✓ Vector DB ready. 14543 docs indexed (dim=768)


In [7]:
# %%
# CELL 6: KnowledgeGraph
from collections import defaultdict

class KnowledgeGraph:
    def __init__(self):
        self.entities = defaultdict(list)
        self.relations = []
        self.context = {}
    def add_entity(self, entity_type, value):
        if value and value.strip() and value not in self.entities[entity_type]:
            self.entities[entity_type].append(value)
    def add_relation(self, subject, relation, obj):
        self.relations.append({'subject': subject, 'relation': relation, 'object': obj})
    def set_context(self, key, value):
        self.context[key] = value
    def to_dict(self):
        return {'entities': dict(self.entities), 'relations': self.relations, 'context': self.context}
    def get_summary(self):
        parts = []
        for etype, vals in self.entities.items():
            if vals:
                parts.append(f"{etype.upper()}: {', '.join(vals[:3])}")
        if self.context.get('situation'):
            parts.append(f"SITUATION: {self.context['situation'][:200]}")
        return ' | '.join(parts) if parts else 'Knowledge graph empty'

print('✓ KnowledgeGraph ready')

✓ KnowledgeGraph ready


In [8]:
# %%
# CELL 8: EntityExtractionAgent
class EntityExtractionAgent:
    def extract(self, text: str):
        out = {'dates': [], 'locations': [], 'values': [], 'items': [], 'parties': []}
        if ner_pipe is not None:
            try:
                ents = ner_pipe(text)
                for e in ents:
                    lab = e.get('entity_group') or e.get('entity')
                    tok = e.get('word') or e.get('entity')
                    if not tok: continue
                    tok = tok.strip()
                    if lab in ['PER']:
                        out['parties'].append(tok)
                    elif lab in ['ORG','LOC']:
                        out['locations'].append(tok)
                    elif lab in ['MISC']:
                        out['items'].append(tok)
            except Exception as e:
                print('NER pipeline error, regex fallback:', e)

        out['dates'] = list(dict.fromkeys(out['dates'] + re.findall(r'\b\d{1,2}[\-/]\d{1,2}[\-/]\d{2,4}\b', text)))[:3]
        months = re.findall(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:,\s*\d{4})?', text, flags=re.I)
        out['dates'] += months
        out['values'] = re.findall(r'\b(?:Rs\.?|₹)\s*[\d,]+\b', text)
        caps = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+){0,2}\b', text)
        out['locations'] += caps
        keywords = ['phone','laptop','car','house','land','jewelry','money','document','agreement','FIR','complaint']
        out['items'] += [kw for kw in keywords if kw.lower() in text.lower()]
        return {k:list(dict.fromkeys(v))[:6] for k,v in out.items()}

print('✓ EntityExtractionAgent ready')

✓ EntityExtractionAgent ready


In [9]:
# %%
# CELL 9: ResearchAgent
class ResearchAgent:
    def __init__(self, documents, index, embedder):
        self.documents = documents
        self.index = index
        self.embedder = embedder
    def research(self, query: str, top_k=6):
        if self.index is None or len(self.documents)==0:
            return [], []
        qvec = self.embedder.encode([query], convert_to_numpy=True).astype('float32')
        distances, indices = self.index.search(qvec, top_k)
        results, dists = [], []
        for pos, idx in enumerate(indices[0]):
            if idx<0 or idx>=len(self.documents): continue
            doc = self.documents[idx].copy()
            doc['excerpt'] = (doc.get('answer') or doc.get('text') or '')[:800].replace('\n',' ')
            results.append(doc)
            dists.append(float(distances[0][pos]))
        return results, dists

print('✓ ResearchAgent ready')

✓ ResearchAgent ready


In [15]:
# ============================================================
# ANALYSIS AGENT — FINAL v5 (FULL ANTI-REPEAT, MISTRAL SAFE)
# ============================================================

import torch, re

class AnalysisAgent:
    def __init__(self, has_llm: bool, max_tokens: int = 650):
        self.has_llm = has_llm and HAS_LLM and LLM_MODEL is not None
        self.max_tokens = max_tokens

    def analyze(self, case_type: str, kg: KnowledgeGraph, docs, dists):

        situation = kg.context.get('situation', '')
        subtype = kg.context.get('criminal_subtype', None)

        fact_text = "\n".join(
            [f"{k.upper()}: {', '.join(v)}" for k, v in kg.entities.items() if len(v)]
        )

        law_text = "\n".join([
            re.sub(r"\s+", " ", f"[{i+1}] {d['excerpt'][:350]}")
            for i, d in enumerate(docs[:3])
        ])

        # ------------------------- PROMPT -------------------------

        prompt = f"""
You are a Senior Indian Criminal Lawyer.

Analyze the situation below with clear legal reasoning.
Follow the exact structure and STOP after "END OF REPORT".

### CLIENT STATEMENT:
{situation}

### FACTS:
{fact_text or 'None'}

### RELEVANT MATERIAL:
{law_text or 'None'}

### CASE TYPE: {case_type.upper()}
### SUBTYPE: {subtype or 'UNKNOWN'}

Write the analysis in the following structure:

<SECTION 1: SUMMARY OF INCIDENT>
Describe exactly what happened.

<SECTION 2: LEGAL CHARACTERIZATION>
Which IPC sections apply and why.

<SECTION 3: LEGAL REASONING>
Explain how the facts satisfy the legal elements.

<SECTION 4: PROSECUTION & DEFENCE>
Write both sides’ arguments.

<SECTION 5: EVIDENCE & STRATEGY>
What evidence is needed + investigation plan.

<SECTION 6: CONCLUSION>
Final senior-lawyer opinion.

END OF REPORT
"""

        # ------------------- INFERENCE --------------------

        if not self.has_llm:
            return {"report": f"[NO LLM AVAILABLE]\nFacts:\n{fact_text}"}

        inputs = LLM_TOKENIZER(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1800
        ).to(DEVICE)

        with torch.no_grad():
            output = LLM_MODEL.generate(
                **inputs,
                max_new_tokens=self.max_tokens,
                temperature=0.3,
                top_p=0.85,
                do_sample=True,
                eos_token_id=LLM_TOKENIZER.eos_token_id,
                pad_token_id=LLM_TOKENIZER.eos_token_id,
            )

        final = LLM_TOKENIZER.decode(output[0], skip_special_tokens=True)
        final = final.replace(prompt.strip(), "").strip()

        # ---------------------- CLEANUP --------------------------

        # 1. STOP at END OF REPORT
        if "END OF REPORT" in final:
            final = final.split("END OF REPORT")[0]

        # 2. HARD ANTI-REPEAT PATCH (kills full duplicated report)
        section_key = "<SECTION 1: SUMMARY OF INCIDENT>"
        parts = final.split(section_key)

        if len(parts) > 2:
            # means:  text + <SECTION1> ... <SECTION6> ... <SECTION1> ...
            final = section_key + parts[1]

        elif len(parts) == 2:
            # normal:     text_before  + <SECTION1> ... END
            final = section_key + parts[1]

        # 3. Remove repeated lines
        lines = final.splitlines()
        seen, cleaned = set(), []
        for line in lines:
            t = line.strip()
            if t and t not in seen:
                cleaned.append(t)
                seen.add(t)

        final = "\n".join(cleaned)

        # 4. Cosmetic: ensure section headings on separate lines
        final = re.sub(r"(<SECTION.*?>)", r"\n\1", final)

        return {"report": final.strip()}

print("✓ AnalysisAgent v5 — FULL NON-REPEATING VERSION Loaded.")


✓ AnalysisAgent v5 — FULL NON-REPEATING VERSION Loaded.


In [11]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [20]:
# =========================================================
# CELL 12 — FIXED CLASSIFIER + MULTI-DOMAIN QUESTION GENERATOR
# =========================================================

class SmartClassifierAgent:
    def __init__(self):
        self.case_keywords = {
            'criminal': ['theft','stolen','robbery','assault','murder','rape','dacoity','fir','police','crime','weapon'],
            'family': ['divorce','marriage','wife','husband','custody','alimony','maintenance','dowry','child','domestic'],
            'property': ['land','plot','boundary','encroachment','tenant','landlord','eviction','property','possession','title'],
            'contract': ['agreement','contract','breach','payment','invoice','loan','debt','default','business']
        }

    def initial_classify(self, query: str):
        q = query.lower()
        scores = {ct: sum(kw in q for kw in kws) for ct, kws in self.case_keywords.items()}
        if max(scores.values()) == 0:
            return "general", 0.3
        best = max(scores, key=scores.get)
        return best, min(scores[best] / 5, 1.0)

    def reconsider(self, initial_type, kg: KnowledgeGraph):
        combined = (kg.get_summary() or "").lower()
        scores = {ct: sum(kw in combined for kw in kws) for ct, kws in self.case_keywords.items()}
        best = max(scores, key=scores.get)

        # Prevent false shift from civil -> criminal
        if initial_type != "criminal" and best == "criminal":
            return initial_type, False

        if scores[best] > scores[initial_type]:
            return best, True

        return initial_type, False


class AdaptiveQuestionGenerator:

    # ---------------- FAMILY ----------------
    FAMILY = [
        "How long have you been married?",
        "Are there children involved?",
        "Has any notice been exchanged?",
        "What relief do you want—divorce, custody, maintenance?",
        "Is domestic violence involved?"
    ]

    # ---------------- PROPERTY ----------------
    PROPERTY = [
        "What type of property is involved?",
        "Who owns the title documents?",
        "Is this an encroachment or boundary dispute?",
        "Since when has this dispute existed?",
        "Any civil suit or police complaint filed?"
    ]

    # ---------------- CONTRACT ----------------
    CONTRACT = [
        "What was the agreement?",
        "Written or oral?",
        "What payment/amount is in dispute?",
        "When did the breach occur?",
        "Do you have evidence (invoices, chats, emails)?"
    ]

    # ---------------- CRIMINAL ----------------
    CRIME_TEMPLATES = {
        'theft': [
            "When was the item last seen?",
            "Where did the theft happen?",
            "What exactly was stolen?",
            "Any CCTV or witnesses?",
            "Was an FIR filed?"
        ],
        'robbery': [
            "Was force or a weapon used?",
            "Any injuries sustained?",
            "What items were taken?",
            "Location of the robbery?",
            "Was police informed?"
        ],
        'assault': [
            "What injuries did you suffer?",
            "Did you receive medical treatment?",
            "Where did the assault occur?",
            "What triggered the fight?",
            "Any witnesses?"
        ],
        'murder': [
            "When did the incident occur?",
            "Where was the body found?",
            "Was a weapon used?",
            "Any eyewitnesses?",
            "What is the relationship between accused and victim?"
        ]
    }

    def detect_crime_subtype(self, text):
        t = text.lower()
        scores = {s: 0 for s in self.CRIME_TEMPLATES.keys()}
        for stype, qs in self.CRIME_TEMPLATES.items():
            for word in qs:
                if word.lower().split()[0] in t:
                    scores[stype] += 1
        return max(scores, key=scores.get)

    def generate_next(self, case_type, kg: KnowledgeGraph, asked):

        # FAMILY
        if case_type == "family":
            for q in self.FAMILY:
                if q not in asked: return q
            return None

        # PROPERTY
        if case_type == "property":
            for q in self.PROPERTY:
                if q not in asked: return q
            return None

        # CONTRACT
        if case_type == "contract":
            for q in self.CONTRACT:
                if q not in asked: return q
            return None

        # CRIMINAL
        subtype = kg.context.get("criminal_subtype") or self.detect_crime_subtype(kg.context.get("situation",""))
        for q in self.CRIME_TEMPLATES.get(subtype, []):
            if q not in asked: return q
        return None


In [23]:
# =========================================================
# CELL 13 — FIXED AGENTIC LEGAL SYSTEM
# =========================================================

class AgenticLegalSystem:
    def __init__(self):
        self.classifier = SmartClassifierAgent()
        self.extractor = EntityExtractionAgent()
        self.qgen = AdaptiveQuestionGenerator()
        self.researcher = ResearchAgent(documents, index, embedder)
        self.analyzer = AnalysisAgent(has_llm=USE_LLM)

    def run_consultation(self, initial_query, interactive=True, max_turns=3):

        print("\n" + "="*60)
        print("AGENTIC LEGAL CONSULTATION")
        print("="*60 + "\n")

        case_type, conf = self.classifier.initial_classify(initial_query)
        print(f"Initial Classification: {case_type} (conf: {conf:.2f})")

        kg = KnowledgeGraph()
        kg.set_context("situation", initial_query)
        kg.set_context("case_type", case_type)

        # ONLY set criminal subtype if criminal case
        if case_type == "criminal":
            kg.set_context("criminal_subtype", self.qgen.detect_crime_subtype(initial_query))
        else:
            kg.set_context("criminal_subtype", None)

        asked = []

        for _ in range(max_turns):
            print("\nKG:", kg.get_summary(), "\n")

            q = self.qgen.generate_next(case_type, kg, asked)
            if not q:
                break

            print("Agent:", q)
            asked.append(q)

            if not interactive:
                continue

            reply = input("You: ").strip()
            ents = self.extractor.extract(reply)
            for et, vals in ents.items():
                for v in vals:
                    kg.add_entity(et, v)

        print("\nInformation gathering complete.\n")

        summary = kg.get_summary()
        docs, dists = self.researcher.research(summary)

        result = self.analyzer.analyze(case_type, kg, docs, dists)
        print(result["report"])
        return result


In [26]:
legal_system = AgenticLegalSystem()

print('='*70)
print(f'CHATLAW – AGENTIC LEGAL SYSTEM (LLM: {LLM_NAME if HAS_LLM else "DETERMINISTIC"})')
print('='*70+'\n')

while True:
    query = input("Your legal query (or 'quit'): ").strip()
    if not query:
        continue
    if query.lower() in ['quit','exit']:
        print('Goodbye.')
        break

    try:
        result = legal_system.run_consultation(query, interactive=True, max_turns=7)
        print('\\n' + result['report'] + '\\n')

        save = input('Save report? (y/n): ').strip().lower()
        if save == 'y':
            fname = f"report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
            with open(fname, 'w') as f:
                f.write(result['report'])
            print(f'✓ Saved to {fname}')
    except Exception as e:
        import traceback
        print(f'\\nError during analysis: {type(e).__name__}: {str(e)[:100]}')
        print('Traceback:')
        traceback.print_exc()
        print('Please try again with a clearer query.\\n')

CHATLAW – AGENTIC LEGAL SYSTEM (LLM: mistralai/Mistral-7B-Instruct-v0.2)

Your legal query (or 'quit'): divorce with my wife due to domestic abuse

AGENTIC LEGAL CONSULTATION

Initial Classification: family (conf: 0.60)

KG: SITUATION: divorce with my wife due to domestic abuse 

Agent: How long have you been married?
You: 7 years

KG: SITUATION: divorce with my wife due to domestic abuse 

Agent: Are there children involved?
You: no

KG: SITUATION: divorce with my wife due to domestic abuse 

Agent: Has any notice been exchanged?
You: yes

KG: SITUATION: divorce with my wife due to domestic abuse 

Agent: What relief do you want—divorce, custody, maintenance?
You: maintanence

KG: SITUATION: divorce with my wife due to domestic abuse 

Agent: Is domestic violence involved?
You: yes

KG: SITUATION: divorce with my wife due to domestic abuse 


Information gathering complete.

<SECTION 1: SUMMARY OF INCIDENT>
The client, a married individual, has stated that they have undergone a divorc